# Cours LangChain TP4

Dans ce TP vous allez voir comment utiliser des chaînes et des agents avec un historique mémoire

In [2]:
!pip install --upgrade --quiet  langchain-openai tavily-python

import os
os.environ['OPENAI_API_KEY']="OPENAI_API_KEY" # À Modifier
os.environ['TAVILY_API_KEY']="TAVILY_API_KEY" # À Modifier

## Exercice 1

Construisez un assistant IA Utilisant l'historique de chat en mémoire pour répondre aux questions
L'assistant devra utiliser une chaine avec un prompt simple 

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Tu es un assistant IA qui réponds aux questions de l'utilisateur en 20 mots ou moins. Tu réponds aux question dans la langue auquel on te l'a posé",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [5]:
print("Bonjour! je suis votre assistant IA. Comment puis-je vous aidez?")
session_id = "abc123"
for i in range(50):#50 echanges
    question = input()
    if question=="clear":
        break
    print(with_message_history.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id }},
    ).content+"\n")

Bonjour! je suis votre assistant IA. Comment puis-je vous aidez?


## Exercice 2

Construisez un agent qui utilisera le tool Tavily pour faire du RAG à travers des recherches sur le net (vous pouvez utiliser l'exemple du cours) l'agent utilisera aussi l'historique de chat en mémoire pour répondre aux questions

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub

# Initialize Tavily and an OpenAI chat model capable of tool calling
tools = [TavilySearchResults(max_results=1)]
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.messages[0]
# Assemble the agent
agent = create_structured_chat_agent(chat, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, handle_parsing_errors=True
)

In [23]:
from pprint import pprint
print("Hi, I'm your AI assistant. How can I help you today?")
chat_history=ChatMessageHistory()
for i in range(50):#50 echanges
    question = input()
    if question=="clear":
        chat_history=[] 
        break
    output = agent_executor.invoke(
        {
            "input": question,
            "chat_history":chat_history.messages,
        }
        )["output"]
    pprint(output)
    chat_history.add_message(HumanMessage(content=question))
    chat_history.add_message(AIMessage(content=output))

Hi, I'm your AI assistant. How can I help you today?


In [13]:
agent_with_message_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

Autre exemple avec `RunnableWithMessageHistory` (L'historique de chat semble un peu capricieux)

In [ ]:
store = {}

print("Bonjour! je suis votre assistant IA. Comment puis-je vous aidez?")
session_id = "def123"
for i in range(50):#50 echanges
    question = input()
    if question=="clear":
        break
    print(agent_with_message_history.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id}},
    )['output']+"\n")